# Direct Solvers

## Kronecker Product - Sparse Solver
### General algorithm
Rewrite $AX+XB=C$ as 
- $\mathcal{A}x = c$

where $\mathcal{A} = I \otimes A + B^* \otimes I$, $x = \text{vec}(X)$ and $c = \text{vec}(C)$

### TU + UT = F
Rewrite as
- $\mathcal{T}u = f$

where $\mathcal{T} = I \otimes T + T \otimes I$, $u = \text{vec}(U)$ and $f = \text{vec}(F)$

In [ ]:
import numpy as np
from scipy.sparse import kron
from scipy.sparse.linalg import spsolve
from scipy.linalg import solve
import time

def kron_prod_dir(T,F):
    n = len(F)
    start_time = time.time()
    F = np.reshape(F,-1)
    A = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n)) #A = I kron T + T^t kron I
    kron_time = time.time() - start_time
    U = spsolve(A, F) #Solve using scipy sparse solver
    #U = solve(A.todense(),F)
    end_time = time.time()
    solve_time = end_time - kron_time - start_time
    total_time = end_time - start_time
    timings = [kron_time, solve_time, total_time]
    U = np.reshape(U, (n,n)) #Reshape U for plotting/computing error
    return U, timings

## Bartels-Stewart (used by SciPy solver)
### General algorithm
1. Compute the Schur forms $A^* = PRP^*$ and $B=QSQ^*$.
2. Solve $R^*Y + YS = \hat{C}$ for $Y$, where $\hat{C} = P^*CQ$.
3. Compute $X=PYQ^*$.

### TU + UT = F
1. Compute the Schur form $T=PRP^*$.
2. Solve $R^*V + VR = \hat{F}$ for $V$, where $\hat{F} =  P^*FP$.
3. Compute $U=PVP^*$.

In [ ]:
import numpy as np
from scipy.linalg import solve_sylvester, schur
import time

def scipy_solver(T, F):
    start_time = time.time()
    U = solve_sylvester(T, T, F)
    end_time = time.time()
    total_time = end_time - start_time
    return U, total_time

def bartels_stewart(T, F):
    n = len(T)
    start_time = time.time()
    R, P = schur(T) #Compute Schur decomposition
    R_trans = R.transpose()
    schur_time = time.time() - start_time
    
    #Solve R^*V + VR = P^*FP for V
    V = np.empty([n,n])
    RHS = np.matmul(np.matmul(P.transpose(),F),P)
    for i in range(0,n):
        for j in range(0,n):                                
            V[i][j] = (RHS[i][j])/(R_trans[i][i]+R[j][j])
    V_time = time.time() - schur_time - start_time
            
    U = np.matmul(np.matmul(P,V),P.transpose()) #Compute U=PVP^*
    end_time = time.time()
    solve_time = end_time - V_time - schur_time - start_time
    total_time = end_time - start_time
    timings = [schur_time, V_time, solve_time, total_time]
    return U, timings

## Similarity Transformation
### General algorithm
1. Compute the eigendecompositions $P^{-1}AP = \text{diag}(\lambda_1, \dots, \lambda_n)$ and $Q^{-1}BQ = \text{diag}(\mu_1, \dots, \mu_m)$:
	1. Compute the eigenpairs of $A$ and $B$
	2. Compute the inverses of $P$ and $Q$
2. Compute the solution $X = P \widetilde{X} Q^{-1}$, where $\widetilde{X}_{ij} = \frac{\widetilde{C}_{ij}}{\lambda_i + \mu_j}$ and $\widetilde{C} = P^{-1}CQ$.

### TU + UT = F
1. Compute the eigendecomposition $P^{-1}TP = \text{diag}(\lambda_1, \dots, \lambda_n)$
	1. Compute the eigenpairs of $T$
	2. Compute the inverse of $P$
2. Compute the solution $U = P \widetilde{U} P^{-1}$, where $\widetilde{U}_{ij} = \frac{\widetilde{F}_{ij}}{\lambda_i + \lambda_j}$ and $\widetilde{F}=P^{-1}FP$

In [ ]:
import numpy as np
from scipy.sparse import diags
import time

def sim_trans(T, F):
    n = len(T)
    start_time = time.time()
    eigvals, eigvecs = np.linalg.eig(T) #Get eigenvalues and eigenvectors of T
    eig_time = time.time() - start_time
    diag = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    P = eigvecs #Set basis matrix P, with columns as the eigenvectors
    P_inv = np.linalg.inv(P)
    inv_time = time.time() - eig_time - start_time
    F_hat = np.matmul(np.matmul(P_inv,F),P) #F_hat = P^-1 * F * P
    U_hat = np.empty([n,n]) #u_hat(i,j) = f(i,j) / eigval(i) + eigval(j)
    for i in range(0,n):
        for j in range(0,n):
            if (eigvals[i]!=0 or eigvals[j]!=0):
                U_hat[i][j] = F_hat[i][j] / (eigvals[i]+eigvals[j])
            else:
                U_hat[i][j] = 0
    U = np.matmul(np.matmul(P,U_hat),P_inv) #U = P * U^hat * P^-1
    end_time = time.time()
    solve_time = end_time - inv_time - eig_time - start_time
    total_time = end_time - start_time
    timings = [eig_time, inv_time, solve_time, total_time]
    return U, timings

def sim_trans_explicit(T, F):
    n = len(T)
    start_time = time.time()
    eigvals = np.empty(n)
    eigvecs = np.empty([n,n])
    i = np.linspace(0,n-1,n)
    j = np.linspace(0,n-1,n)
    I, J = np.meshgrid(i, j, indexing='ij')
    eigvals = (T[0][0] - T[0][0]*np.cos((i+1)*np.pi/(n+1)))
    eigvecs = np.sqrt(2/(n+1))*np.sin(((I+1)*(J+1)*np.pi)/(n+1))
    eig_time = time.time() - start_time
    diag = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    P = eigvecs #Set basis matrix P, with columns as the eigenvectors
    P_trans = P.transpose()
    inv_time = time.time() - eig_time - start_time
    F_hat = np.matmul(np.matmul(P_trans,F),P) #F_hat = P^-1 * F * P
    
    U_hat = np.empty([n,n]) #u_hat(i,j) = f(i,j) / eigval(i) + eigval(j)
    for i in range(0,n):
        for j in range(0,n):
            if (eigvals[i]!=0 or eigvals[j]!=0):
                U_hat[i][j] = F_hat[i][j] / (eigvals[i]+eigvals[j])
            else:
                U_hat[i][j] = 0

    U = np.matmul(np.matmul(P,U_hat),P_trans) #U = P * U^hat * P^-1
    end_time = time.time()
    solve_time = end_time - inv_time - eig_time - start_time
    total_time = end_time - start_time
    timings = [eig_time, inv_time, solve_time, total_time]
    return U, timings

## Shifted System
### General algorithm
1. Compute $B = WSW^{-1}$, where $S = \text{diag}(s_1, \dots, s_n)$ are the eigenvalues of $B$ and the columns of $W$ are the eigenvectors of $B$
    1. Compute eigenpairs of $B$
    2. Compute inverse of $W$
2. For $i=1$ to $n$, solve the system $(A+s_i I)(\hat{X})_i = (\hat{C})_i$, where $\hat{C} = CW$
3. Compute solution $X = \hat{X}W^{-1}$

where $(\hat{X})_i$ denotes the $i^{\text{th}}$ column of $\hat{X}$.

### TU + UT = F
1. Compute $T = WSW^{-1}$, where $S = \text{diag}(s_1, \dots, s_n)$ are the eigenvalues of $T$ and the columns of $W$ are the eigenvectors of $T$
    1. Compute eigenpairs of $T$
    2. Compute inverse of $W$
2. For $i=1$ to $n$, solve the system $(T+s_i I)(\hat{U})_i = (\hat{F})_i$, where $\hat{F} = FW$
3. Compute solution $U = \hat{U}W^{-1}$

In [ ]:
import numpy as np
import time
from scipy.sparse.linalg import spsolve

def shifted_sys(T, F):
    n = len(T)
    start_time = time.time()
    eigvals = np.empty(n)
    eigvecs = np.empty([n,n])
    i = np.linspace(0,n-1,n)
    j = np.linspace(0,n-1,n)
    I, J = np.meshgrid(i, j, indexing='ij')
    eigvals = (T[0][0] - T[0][0]*np.cos((i+1)*np.pi/(n+1)))
    eigvecs = np.sqrt(2/(n+1))*np.sin(((I+1)*(J+1)*np.pi)/(n+1))
    eig_time = time.time() - start_time
    S = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    W = eigvecs
    W_inv = W.transpose() #W is Hermitian so W^-1 = W^T
    inv_time = time.time() - eig_time - start_time
    F_hat = np.matmul(F,W)
    U_hat = np.zeros([n,n])
    
    for i in range(0,n):
        A = T + S[i][i]*np.eye(n)
        U_hat[:,i] = spsolve(A,F_hat[:,i]) #use cg(A,F_hat[:,i])[0] for iterative version
    
    solve_time = time.time() - inv_time - eig_time - start_time
    U = np.matmul(U_hat, W_inv)
    end_time = time.time()
    U_time = end_time - solve_time - inv_time - eig_time - start_time
    total_time = end_time - start_time
    timings = [eig_time, inv_time, solve_time, U_time, total_time]
    return U, timings

In [ ]:
import numpy as np

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

In [ ]:
from scipy.sparse import diags

#Define parameters
n = 1000 #number of internal nodes in each direction (number of unknowns)
h = 1/(n+1) #step size

U = np.zeros([n,n])

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Define F 
F = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y)  

#Define tridiagonal matrix T
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute exact solution for comparison
U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y)

#Solve system
U, total_time = sim_trans_explicit(T,F)
err_inf, err_sq = compute_err(U, U_exact)
print('Time taken:', total_time)
print('Error inf:', err_inf, '\nError sq:', err_sq)